# 01. Extração dos dados

Vou buscar e realizar uma extração dos dados dos dados relacionados aos feriados no mundo todo [nesse site](https://www.timeanddate.com/holidays/), e depois buscar e baixar dados relacionados a índices socioeconômicos, criando assim uma base sólida para meu estudo.


## Inicialização

Chamando bibliotecas a serem utilizadas nesse _notebook_


In [124]:
import numpy as np
import pandas as pd
import itertools as itt
import requests
import re
import time
from bs4 import BeautifulSoup
from pprint import pprint

Funções a serem utilizadas nesse notebook

In [125]:
# extração das informações referentes ao feriado

def extract_holiday(link):
    # para evitar do site barrar nossos múltiplos acessos rápidos, vou simular um tempo aleatório
    time.sleep(1)

    # para nos situar, vamos printar a página a qual estamos extraindo os dados
    print(link)

    r = requests.get(link)
    page = BeautifulSoup(r.content)
    table = page.find("tbody")

    # extraindo lista com as datas dos feriados
    data = table.find_all("th", class_= "nw")
    data = [i.string for i in data]

    # lista com os nomes dos feriados
    nome = table.find_all("a")
    nome = [i.string for i in nome]

    # lista com o tipo de feriado
    # algumas páginas possuem 4 colunas e outras 5 colunas, vamos criar
    # uma condicional para passar por todos os casos:

    # criando condição
    cond = page.find("tr")
    # contando número de colunas
    tam = len(cond.find_all("th"))
    if tam == 4:
        tipo = table.find_all("td")
        tipo = tipo[2::3]
        tipo = [i.string for i in tipo]
        
    else:
        tipo = table.find_all("td")
        tipo = tipo[2::4]
        tipo = [i.string for i in tipo]

    # identificação do país
    pais = link.split('/')
    pais = pais[4]
    pais = itt.repeat(pais, len(tipo))

    df = pd.DataFrame({ "país" : pais,
                        "data" : data,
                        "nome": nome,
                        "tipo" : tipo})
    return df

## Extração

### Dados relacionados a feriados
Realizando a extração dos dados referentes aos feriados, para isso, vou realizar um _web scrapping_ no site https://www.timeanddate.com/holidays/ por meio da biblioteca *BeautifulSoup4*. 

Inicialmente vamos pegar todos os países presentes no site e salvar em uma lista.

In [126]:
r = requests.get("https://www.timeanddate.com/holidays/?allcountries")

# Convertendo a página para o bs
holidays = BeautifulSoup(r.content)

# endereço em html de onde se encontra a lista de países disponíveis no site
holidays = holidays.find("article", class_= "category-list")
holidays = holidays.find_all("a", attrs={'href': True})

# coletando apenas as strings com nomes dos países
aux = [i.string for i in holidays]
aux[1:10] # visualizando os primeiros valores da lista


['Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia']

In [127]:
aux

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 None,
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo Democratic Republic',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Falkland Islands',
 'Faroe Islands',
 'Fiji',
 'Finland',


In [128]:
# criando uma lista com todos os países

hyperlink = "https://www.timeanddate.com/holidays/"

# transformando o tipo da lista auxiliar em string para realizar a concatenação do link
aux = [str(i) for i in aux if type(i) != NoneType]

# update: casos especiais
## Apesar de se tratar de um mal exemplo de "hard coding", não houve muita
## escapatória se não substituir "a força" os casos em que o nome do país difere
## do nome presente na URL do site.

aux = [i.replace("Bosnia and Herzegovina", "bosnia") for i in aux]
aux = [i.replace("Trinidad and Tobago", "Trinidad") for i in aux]
aux = [i.replace("Cabo Verde", "cape-verde") for i in aux] 
aux = [i.replace("Vatican City (Holy See)", "vatican-city-state") for i in aux]
aux = [i.replace("Comoros", "comores") for i in aux] 
aux = [i.replace("Congo Democratic Republic", "dr-congo") for i in aux]
aux = [i.replace("Congo", "republic-of-the-congo") for i in aux] 
aux = [i.replace("Cote d'Ivoire", "ivory-coast") for i in aux]  
aux = [i.replace("Czechia", "czech") for i in aux]
aux = [i.replace("Curaçao", "curacao") for i in aux] 
aux = [i.replace("East Timor", "timor-leste") for i in aux]  
aux = [i.replace("Equatorial Guinea", "guineaecuatorial") for i in aux]
aux = [i.replace("Eswatini", "swaziland") for i in aux]  
aux = [i.replace("North Macedonia", "macedonia") for i in aux]  
aux = [i.replace("St. Barts", "saint-barthelemy") for i in aux]   
aux = [i.replace("The Bahamas", "bahamas") for i in aux] 
aux = [i.replace("United Kingdom", "uk") for i in aux] 
aux = [i.replace("United States", "us") for i in aux] 
aux = [i.replace("US Virgin Islands", "united-states-virgin-islands") for i in aux]

# em links, letras maiúsculas importam, e nossos dados possuem letras maiúscula no inicio do nome do país
# já no hyperlink referente a esse site, a letra é minúscula, então façamos as mudanças necessárias
aux = [i.lower() for i in aux]

# substituindo espaço em branco por traço para o acesso dos links
aux = [i.replace(" ", "-") for i in aux]

link_paginas = ["https://www.timeanddate.com/holidays/" + i + "/" for i in aux]
link_paginas[0:5]

['https://www.timeanddate.com/holidays/afghanistan/',
 'https://www.timeanddate.com/holidays/albania/',
 'https://www.timeanddate.com/holidays/algeria/',
 'https://www.timeanddate.com/holidays/american-samoa/',
 'https://www.timeanddate.com/holidays/andorra/']

os sites foram checados e parecem funcionar bem, agora vamos ao que interessa: pegar os valores de:

- data do feriado
- nome
- tipo de feriado

e transferir para um dataframe

In [129]:
df_holidays = pd.DataFrame()

for i in link_paginas:
  df_holidays = pd.concat([df_holidays, extract_holiday(i)])

https://www.timeanddate.com/holidays/afghanistan/
https://www.timeanddate.com/holidays/albania/
https://www.timeanddate.com/holidays/algeria/
https://www.timeanddate.com/holidays/american-samoa/
https://www.timeanddate.com/holidays/andorra/
https://www.timeanddate.com/holidays/angola/
https://www.timeanddate.com/holidays/anguilla/
https://www.timeanddate.com/holidays/antigua-and-barbuda/
https://www.timeanddate.com/holidays/argentina/
https://www.timeanddate.com/holidays/armenia/
https://www.timeanddate.com/holidays/aruba/
https://www.timeanddate.com/holidays/australia/
https://www.timeanddate.com/holidays/austria/
https://www.timeanddate.com/holidays/azerbaijan/
https://www.timeanddate.com/holidays/bahrain/
https://www.timeanddate.com/holidays/bangladesh/
https://www.timeanddate.com/holidays/barbados/
https://www.timeanddate.com/holidays/belarus/
https://www.timeanddate.com/holidays/belgium/
https://www.timeanddate.com/holidays/belize/
https://www.timeanddate.com/holidays/benin/
https

Dados extraídos com sucesso !!

In [130]:
df_holidays

,país,data,nome,tipo
0,afghanistan,15 de Fev,Liberation Day,Public Holiday
1,afghanistan,20 de Mar,March Equinox,Season
2,afghanistan,21 de Mar,Nauruz,Observance
3,afghanistan,2 de Abr,First Day of Ramadan,Public Holiday
4,afghanistan,28 de Abr,Afghan Victory Day,Public Holiday
...,...,...,...,...
18,zimbabwe,22 de Dez,Unity Day,Public Holiday
19,zimbabwe,25 de Dez,Christmas Day,Public Holiday
20,zimbabwe,26 de Dez,Christmas Day observed,Public Holiday
21,zimbabwe,26 de Dez,Boxing Day,Public Holiday


In [136]:
# vamos exportar esse dataframe em formato de CSV 

df_holidays.to_csv("./feriados.csv", index = False)

### Dados relacionados a índices socioeconômicos

Para os dados contendo indices socioeconômicos, fiz um download [neste link](https://hdr.undp.org/data-center/human-development-index#/indicies/HDI), diretamente do site da Organização das Nações Unidas.

Agora vamos passar para a concatenação e tratamento dos dados presentes. Esse processo se encontrará na pasta 02. Tratamento dos dados nesse mesmo projeto de GitHub.